In [1]:
from IPython import display
import math
import pandas as pd
import numpy as np

from pyspark.sql import SQLContext
from pyspark import SparkContext

from pyspark.sql.types import *

In [2]:
sc =SparkContext()
sqlContext = SQLContext(sc)

21/12/06 11:21:16 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.29.178 instead (on interface wlo1)
21/12/06 11:21:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/06 11:21:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/codevardhan/dev/mental_health_analysis/env/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [11]:
customSchema = StructType([
    StructField("clean_text", StringType()), 
    StructField("category", StringType())])

In [15]:
filename1 = 'data/tweets.csv'
filename2 = 'data/reddit.csv'

In [16]:
df1 = sqlContext.read.format("csv").option("header", "true").schema(customSchema).load(filename1)
df1.count()

4175

In [17]:
df2 = sqlContext.read.format("csv").option("header", "true").schema(customSchema).load(filename2)
df2.count()

989

In [18]:
df = df1.union(df2)
df.count() 

5164

In [19]:
data = df.na.drop(how='any')
data.show(5)

+----------+--------------------+
|clean_text|            category|
+----------+--------------------+
|         0|                none|
|         1|worry others focu...|
|         2|denial anger barg...|
|         3|really went etern...|
|         4|2021 chez moi tra...|
+----------+--------------------+
only showing top 5 rows



21/12/06 11:22:26 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv


In [20]:
data.printSchema()

root
 |-- clean_text: string (nullable = true)
 |-- category: string (nullable = true)



In [21]:
from pyspark.sql.functions import col

data.groupBy("category").count().orderBy(col("count").desc()).show()

21/12/06 11:22:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv
21/12/06 11:22:28 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/reddit.csv


+--------------------+-----+
|            category|count|
+--------------------+-----+
|                none|  143|
|          depression|   29|
|                  -1|    7|
|                   1|    7|
|                   "|    6|
|please help sign ...|    4|
|     depression real|    4|
|post concert depr...|    4|
|someone suffered ...|    3|
|     depression joke|    3|
|  smiling depression|    2|
|post concert depr...|    2|
|make sure take ca...|    2|
|spiritual awakeni...|    2|
|someone suffered ...|    2|
|read carefully pl...|    2|
|tropical depressi...|    2|
|denial anger barg...|    2|
|      visit exciting|    2|
| seasonal depression|    2|
+--------------------+-----+
only showing top 20 rows



*Model Pipeline*

In [22]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="clean_text", outputCol="words", pattern="\\W")

# stop words
add_stopwords = ["http","https","amp","rt","t","c","the"] 

stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=20000, minDF=5)

In [23]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
dataset.show(5)

21/12/06 11:22:32 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv
21/12/06 11:22:32 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/reddit.csv
21/12/06 11:22:33 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/reddit.csv
21/12/06 11:22:33 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv


+----------+--------------------+-----+--------+----------+------+
|clean_text|            category|words|filtered|  features| label|
+----------+--------------------+-----+--------+----------+------+
|         0|                none|  [0]|     [0]|(39,[],[])|   0.0|
|         1|worry others focu...|  [1]|     [1]|(39,[],[])|4669.0|
|         2|denial anger barg...|  [2]|     [2]|(39,[],[])|1364.0|
|         3|really went etern...|  [3]|     [3]|(39,[],[])|3600.0|
|         4|2021 chez moi tra...|  [4]|     [4]|(39,[],[])| 664.0|
+----------+--------------------+-----+--------+----------+------+
only showing top 5 rows



21/12/06 11:22:34 WARN DAGScheduler: Broadcasting large task binary with size 1111.3 KiB
21/12/06 11:22:34 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv


*Partition Training & Test sets & Model Training and Evaluation*

In [24]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

21/12/06 11:22:36 WARN DAGScheduler: Broadcasting large task binary with size 1126.6 KiB
21/12/06 11:22:36 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv
21/12/06 11:22:36 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/reddit.csv


Training Dataset Count: 3624


21/12/06 11:22:36 WARN DAGScheduler: Broadcasting large task binary with size 1126.6 KiB
21/12/06 11:22:36 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv
21/12/06 11:22:36 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/reddit.csv


Test Dataset Count: 1527


In [25]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0).select("clean_text","category","probability","label","prediction")\
.orderBy("probability", ascending=False).show(n = 10, truncate = 30)

21/12/06 11:22:38 WARN DAGScheduler: Broadcasting large task binary with size 1133.3 KiB
21/12/06 11:22:38 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv
21/12/06 11:22:38 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/reddit.csv
21/12/06 11:22:38 WARN DAGScheduler: Broadcasting large task binary with size 1134.9 KiB
21/12/06 11:22:38 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv
21/12/06 11:22:38 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Heade

+----------+------------------------------+------------------------------+------+----------+
|clean_text|                      category|                   probability| label|prediction|
+----------+------------------------------+------------------------------+------+----------+
|       101| bff who have known since 4...|[0.027181566113153158,0.016...| 161.0|       0.0|
|       102| hate being alive hate have...|[0.027181566113153158,0.016...| 320.0|       0.0|
|       110| boyfriend super excited ab...|[0.027181566113153158,0.016...| 164.0|       0.0|
|       111| feel like struggle articul...|[0.027181566113153158,0.016...| 264.0|       0.0|
|       114| won freedom and just becom...|[0.027181566113153158,0.016...| 616.0|       0.0|
|       116| year old male had horrible...|[0.027181566113153158,0.016...| 625.0|       0.0|
|       118| been trying for years find...|[0.027181566113153158,0.016...| 159.0|       0.0|
|       119| useless take space one nee...|[0.027181566113153158,0.016

In [29]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

21/12/06 11:23:00 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
21/12/06 11:23:00 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv
21/12/06 11:23:00 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/reddit.csv


0.0016466243572194424

In [27]:

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.2-bin-hadoop2.7"

In [28]:
from pyspark.ml.feature import HashingTF, IDF

hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=30000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])

pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)

(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)

predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("clean_text","category","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

21/12/06 11:22:56 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv
21/12/06 11:22:56 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/reddit.csv
21/12/06 11:22:57 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/reddit.csv
21/12/06 11:22:57 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 3, schema size: 2
CSV file: file:///home/codevardhan/dev/mental_health_analysis/data/tweets.csv
21/12/06 11:22:57 WARN DAGScheduler: Broadcasting large task binary with

Py4JJavaError: An error occurred while calling o699.fit.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.ml.linalg.DenseMatrix$.zeros(Matrices.scala:491)
	at org.apache.spark.ml.classification.LogisticRegression.createInitialSolution(LogisticRegression.scala:834)
	at org.apache.spark.ml.classification.LogisticRegression.$anonfun$train$1(LogisticRegression.scala:613)
	at org.apache.spark.ml.classification.LogisticRegression$$Lambda$4015/0x00000001016e6840.apply(Unknown Source)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at org.apache.spark.ml.util.Instrumentation$$$Lambda$4016/0x00000001016e7440.apply(Unknown Source)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.classification.LogisticRegression.train(LogisticRegression.scala:495)
	at org.apache.spark.ml.classification.LogisticRegression.train(LogisticRegression